## Credit Score Classification

### Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Settings

In [ ]:
# Display every column

pd.set_option('display.max_columns', None)

### Data exploration & cleaning

In [ ]:
# Load the dataset

df = pd.read_csv(filepath_or_buffer='./Data/train.csv', low_memory=False)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# Check for duplicate entries in the dataset

df.duplicated().any()

In [ ]:
# Get unique values from every column

columns = df.columns
for column in columns:
    print(f'{column}: {df[column].unique()}\n')

In [ ]:
# Find errors in the SSN column

SSN_regex_pattern = r'^\d{3}-\d{2}-\d{4}$'

invalid_ssns = ~df['SSN'].str.match(pat=SSN_regex_pattern, na=True)
invalid_ssns.sum()

In [ ]:
# Preview incorrect SSN entries

df.loc[invalid_ssns, 'SSN']

In [ ]:
# View all the info for records with incorrect SSNs

invalid_ssns_info = df.loc[invalid_ssns, :]
invalid_ssns_info

#### Removing errors in data

In [ ]:
# Remove incorrect entries from the SSN column

invalid_ssn_customer_IDs = invalid_ssns_info['Customer_ID'].unique()

for customer in invalid_ssn_customer_IDs:
    # Filter rows for the current Customer_ID
    customer_rows = df[df['Customer_ID'] == customer]
    # Extract valid SSN values if found
    valid_ssns = customer_rows['SSN'].dropna().unique()
    # Update the SSN for the customer with a valid one, if found
    if len(valid_ssns) > 0:
        df.loc[df['Customer_ID'] == customer, 'SSN'] = valid_ssns[0]

In [ ]:
df[df['SSN'] == '#F%$D@*&8']

In [ ]:
df[df['Customer_ID'] == 'CUS_0x132f']

In [ ]:
# Count the amount of customers with a missing SSN

len(df.loc[df['SSN'] == '#F%$D@*&8', 'Customer_ID'].unique())

In [ ]:
# Remove customers with an invalid SSN

df = df[df['SSN'] != '#F%$D@*&8']

In [ ]:
# Remove leading and trailing underscores from the data

df['Age'] = df['Age'].str.strip(to_strip='_')
df['Annual_Income'] = df['Annual_Income'].str.strip(to_strip='_')
df['Num_of_Loan'] = df['Num_of_Loan'].str.strip(to_strip='_')
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].str.strip(to_strip='_')
df['Outstanding_Debt'] = df['Outstanding_Debt'].str.strip(to_strip='_')
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].str.strip(to_strip='_')

for column in columns:
    print(f'{column}: {df[column].unique()}\n')

In [ ]:
# Remove the underscore entries from the Occupation column

df.loc[df['Occupation'] == '_______']

df = df.loc[df['Occupation'] != '_______']

# Remove the underscore entries from the Changed_Credit_Limit column

df.loc[df['Changed_Credit_Limit'] == '_']

df = df.loc[df['Changed_Credit_Limit'] != '_']

# Remove the underscore entries from the Credit_Mix column

df.loc[df['Credit_Mix'] == '_']

df = df.loc[df['Credit_Mix'] != '_']

In [ ]:
# Remove invalid entries (negative values, overestimated values)

# Fill in incorrect ages

incorrect_ages = df.loc[
    ~(df['Age'].astype(str).str.match(r'^\d{2,3}$'))
    | (df['Age'].astype(str).str.contains('-'))
    | (df['Age'].astype(int) > 120)
]

df_dropped_ages = df[~df.index.isin(incorrect_ages.index)]
incorrect_ages

In [ ]:
# Validate the age on other records with the same Customer_ID

df.loc[df['Customer_ID'] == 'CUS_0x6a1b']

In [ ]:
# Calculate the mean of valid age records and update the incorrect Age column records

for customer_id in incorrect_ages['Customer_ID'].unique():
    # Get all age inputs for the customer with a specific Customer_ID, excluding incorrect ages
    correct_ages = df.loc[(df['Customer_ID'] == customer_id) & ~(df.index.isin(incorrect_ages.index)), 'Age']

    # Convert correct ages to numeric values
    correct_ages_numeric = pd.to_numeric(correct_ages, errors='coerce')

    # Calculate and round the mean of correct ages
    mean_age = np.nanmean(correct_ages_numeric)
    mean_age = mean_age.round().astype(int)

    # Update incorrect ages in the original dataframe
    mask = (df['Customer_ID'] == customer_id) & (df.index.isin(incorrect_ages.index))
    df.loc[mask, 'Age'] = mean_age

In [ ]:
df.loc[df['Customer_ID'] == 'CUS_0x4080']

In [ ]:
# Recheck incorrect ages

incorrect_ages = df.loc[
    ~(df['Age'].astype(str).str.match(r'^\d{2,3}$'))
    | (df['Age'].astype(str).str.contains('-'))
    | (df['Age'].astype(int) > 120)
]

df_dropped_ages = df[~df.index.isin(incorrect_ages.index)]
incorrect_ages

In [ ]:
df.loc[df['Name'] == 'John McCrankb']

In [ ]:
# Remove the customer with the Customer_ID of CUS_0x19cc from the data, as the age could not be corrected

df = df[df['Customer_ID'] != 'CUS_0x19cc']

In [ ]:
# Remove incorrect entries from the Payment_Behaviour column

df.loc[df['Payment_Behaviour'] == '!@9#%8']

In [ ]:
df.loc[df['Customer_ID'] == 'CUS_0x95ee']

In [ ]:
df_replace_payment_behavior = df['Payment_Behaviour'].replace(to_replace='!@9#%8', value=np.nan)

In [ ]:
df_replace_payment_behavior.isna().sum()

In [ ]:
# Fill NaN values with either frontfill or backfill

df_replace_payment_behavior.ffill(inplace=True)
df_replace_payment_behavior.bfill(inplace=True)
df_replace_payment_behavior.isna().sum()

In [ ]:
# Move corrected column back into the original dataset

df['Payment_Behaviour'] = df_replace_payment_behavior
df['Payment_Behaviour']

In [ ]:
df.isnull().sum()

In [ ]:
# Remove incorrect entries from the Monthly_Inhand_Salary column

df.loc[df['Monthly_Inhand_Salary'].isnull()]

In [ ]:
df.loc[df['Customer_ID'] == 'CUS_0xa053']

In [ ]:
# Create a set of customer IDs with NaN salary values

nan_customer_ids = df.loc[df['Monthly_Inhand_Salary'].isnull(), 'Customer_ID'].unique()

# Calculate the mean of valid in-hand salary records and update the missing values

for customer_id in nan_customer_ids:
    # Filter NaN salary values for each Customer_ID
    mask_nan_salaries = (df['Customer_ID'] == customer_id) & (df['Monthly_Inhand_Salary'].isnull())
    # Get salaries and calculate the mean value
    correct_salaries = df.loc[(df['Customer_ID'] == customer_id) & ~(df['Monthly_Inhand_Salary'].isnull()), 'Monthly_Inhand_Salary']
    mean_salaries = np.nanmean(correct_salaries)

    # Fill in missing salaries
    df.loc[mask_nan_salaries, 'Monthly_Inhand_Salary'] = mean_salaries

In [ ]:
df.loc[df['Customer_ID'] == 'CUS_0x2084']

In [ ]:
df.describe(include='object').T

In [ ]:
df.describe().T

In [ ]:
df.loc[df['Monthly_Balance'] == '__-333333333333333333333333333__']

In [ ]:
df.loc[df['SSN'] == '264-53-1943']

In [ ]:
# Fill the Monthly_Balance column with mean values derived from other rows

# Convert Monthly_Balance entries to numeric values

df['Monthly_Balance'] = pd.to_numeric(df['Monthly_Balance'], errors='coerce')

incorrect_monthly_balance_ssns = df.loc[df['Monthly_Balance'].isna(), 'SSN'].unique()

# Calculate the mean of valid monthly balance records and update the incorrect values

for customer_ssn in incorrect_monthly_balance_ssns:
    # Filter incorrect balances for each customer SSN
    mask_nan_balances = (df['SSN'] == customer_ssn) & (df['Monthly_Balance'].isna())
    # Get other balances and calculate the mean value
    correct_balances = df.loc[(df['SSN'] == customer_ssn) & ~(df['Monthly_Balance'].isna()), 'Monthly_Balance']
    mean_balances = np.nanmean(correct_balances)

    # Fill in incorrect balances
    df.loc[mask_nan_balances, 'Monthly_Balance'] = mean_balances

In [ ]:
# Check if the values were successfully replaced

df.loc[df['Monthly_Balance'].isna()]

In [ ]:
df.loc[df['SSN'] == '537-73-6299']

In [ ]:
# Drop the rows which were unable to be filled

df = df[df['SSN'] != '537-73-6299']

In [ ]:
df.isna().sum()

In [ ]:
# Update or remove the Amount_invested_monthly column entries

df.loc[df['Amount_invested_monthly'].isnull()]

In [ ]:
df.loc[df['Customer_ID'] == 'CUS_0x5407']

In [ ]:
# Convert Amount_invested_monthly entries to numeric values

df['Amount_invested_monthly'] = pd.to_numeric(df['Amount_invested_monthly'], errors='coerce')

null_invested_monthly_customers = df.loc[df['Amount_invested_monthly'].isnull(), 'Customer_ID'].unique()

for customer in null_invested_monthly_customers:
    # Filter NaN values
    mask_nan_amounts = (df['Customer_ID'] == customer) & (df['Amount_invested_monthly'].isna())
    # Get correct values for the Amount_invested_monthly
    correct_amounts = df.loc[(df['Customer_ID'] == customer) & ~(df['Amount_invested_monthly'].isna()), 'Amount_invested_monthly']
    mean_amounts = np.nanmean(correct_amounts)

    # Fill NaN values
    df.loc[mask_nan_amounts, 'Amount_invested_monthly'] = mean_amounts

In [ ]:
# Check update results

df.loc[df['Amount_invested_monthly'].isnull()]

In [ ]:
df.loc[df['Name'] == 'Jonathan Spicerr']

In [ ]:
df.isna().sum()

In [ ]:
# Remove the entry which was unable to be updated

df = df[df['SSN'] != '336-19-9190']

In [ ]:
# Update or remove the Credit_History_Age entries

df.loc[df['Credit_History_Age'].isna()]

In [ ]:
df.loc[df['Customer_ID'] == 'CUS_0xd40']

In [ ]:
df['Credit_History_Age'].isna().sum()

In [ ]:
nan_credit_age_customer_ids = df.loc[df['Credit_History_Age'].isna(), 'Customer_ID'].unique()

for customer_id in nan_credit_age_customer_ids:
    # Filter rows for the specified customer_id
    customer_data = df.loc[df['Customer_ID'] == customer_id, 'Credit_History_Age']

    # Forward fill NaN values
    customer_data_filled = customer_data.ffill()

    # Backward fill remaining NaN values
    customer_data_filled = customer_data_filled.bfill()

    # Update the original DataFrame with the filled and updated values
    df.loc[df['Customer_ID'] == customer_id, 'Credit_History_Age'] = customer_data_filled

In [ ]:
# Validate the changes made

df.loc[df['Customer_ID'] == 'CUS_0xd40', 'Credit_History_Age']

In [ ]:
df.loc[df['Credit_History_Age'].isna()]

In [ ]:
# Remove the entries which were unable to be filled

df = df[df['Customer_ID'] != 'CUS_0xbc14']

In [ ]:
df.isna().sum()